In [1]:
!git clone https://github.com/zeroix15/Twitter_Sarcasm_Detections.git

Cloning into 'Twitter_Sarcasm_Detections'...
remote: Enumerating objects: 186, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 186 (delta 107), reused 170 (delta 91), pack-reused 0 (from 0)
Receiving objects: 100% (186/186), 11.24 MiB | 9.50 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [2]:
!pip install transformers==4.40.2
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 24.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 M

In [3]:
# Import Modules
import csv
import urllib.request

import pandas as pd

# from simpletransformers.classification import ClassificationModel
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# # Download dataset
# train_url="https://raw.githubusercontent.com/zeroix15/Twitter_Sarcasm_Detections/main/ptacek/train_cleaned.csv"
# validation_url="https://raw.githubusercontent.com/zeroix15/Twitter_Sarcasm_Detections/main/ptacek/dev.csv"
# test_url="https://raw.githubusercontent.com/zeroix15/Twitter_Sarcasm_Detections/main/ptacek/test_cleaned.csv"


# #rain_url="https://raw.githubusercontent.com/zeroix15/Twitter_Sarcasm_Detections/main/text/train.csv"
# #validation_url="https://raw.githubusercontent.com/zeroix15/Twitter_Sarcasm_Detections/main/text/dev.csv"
# #test_url="https://raw.githubusercontent.com/zeroix15/Twitter_Sarcasm_Detections/main/text/test.csv"

In [6]:
df = pd.read_csv("/content/drive/My Drive/AlifResearch/sarcasm_v2/GEN-sarc-notsarc.csv")
df1 = pd.read_csv("/content/drive/My Drive/AlifResearch/sarcasm_v2/HYP-sarc-notsarc.csv")
df2 = pd.read_csv("/content/drive/My Drive/AlifResearch/sarcasm_v2/RQ-sarc-notsarc.csv")

In [7]:
# Concatenate vertically
df = pd.concat([df, df1, df2], ignore_index=True)
df

,class,id,text
0,notsarc,1,"If that's true, then Freedom of Speech is doom..."
1,notsarc,2,Neener neener - is it time to go in from the p...
2,notsarc,3,"Just like the plastic gun fear, the armour pie..."
3,notsarc,4,So geology is a religion because we weren't he...
4,notsarc,5,Well done Monty. Mark that up as your first ev...
...,...,...,...
9381,sarc,1698,"Tell me genius, how is me accurately and corre..."
9382,sarc,1699,So you think it is a good idea for public scho...
9383,sarc,1700,"Now settle down charlie, and try to think rati..."
9384,sarc,1701,The VPC has a political agenda. The FBI? That ...


In [8]:
df= df.drop('id', axis= 1)
df

,class,text
0,notsarc,"If that's true, then Freedom of Speech is doom..."
1,notsarc,Neener neener - is it time to go in from the p...
2,notsarc,"Just like the plastic gun fear, the armour pie..."
3,notsarc,So geology is a religion because we weren't he...
4,notsarc,Well done Monty. Mark that up as your first ev...
...,...,...
9381,sarc,"Tell me genius, how is me accurately and corre..."
9382,sarc,So you think it is a good idea for public scho...
9383,sarc,"Now settle down charlie, and try to think rati..."
9384,sarc,The VPC has a political agenda. The FBI? That ...


In [9]:
df['class'] = df['class'].replace({'sarc': 1, 'notsarc': 0})
df['class'] = df['class'].astype(int)
df

,class,text
0,0,"If that's true, then Freedom of Speech is doom..."
1,0,Neener neener - is it time to go in from the p...
2,0,"Just like the plastic gun fear, the armour pie..."
3,0,So geology is a religion because we weren't he...
4,0,Well done Monty. Mark that up as your first ev...
...,...,...
9381,1,"Tell me genius, how is me accurately and corre..."
9382,1,So you think it is a good idea for public scho...
9383,1,"Now settle down charlie, and try to think rati..."
9384,1,The VPC has a political agenda. The FBI? That ...


In [10]:
cols = list(df.columns)
cols[0], cols[1] = cols[1], cols[0]  # Swap the positions of the first two columns
df = df[cols]
df

,text,class
0,"If that's true, then Freedom of Speech is doom...",0
1,Neener neener - is it time to go in from the p...,0
2,"Just like the plastic gun fear, the armour pie...",0
3,So geology is a religion because we weren't he...,0
4,Well done Monty. Mark that up as your first ev...,0
...,...,...
9381,"Tell me genius, how is me accurately and corre...",1
9382,So you think it is a good idea for public scho...,1
9383,"Now settle down charlie, and try to think rati...",1
9384,The VPC has a political agenda. The FBI? That ...,1


In [11]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': False,
    'max_seq_length': 40,
    'learning_rate': 0.00001,
    'weight_decay': 0.01,
    'warmup_ratio': 0.2,
    'max_grad_norm': 1.0,
    'num_train_epochs': 8,
    'train_batch_size': 32, #32
    'save_model_every_epoch': False,
    'fp16': True,
    'manual_seed': 128,
    'output_dir': '/outputs/',
    'evaluate_during_training': True,
    'evaluate_during_training_verbose': True
}

In [12]:
from sklearn.model_selection import train_test_split

train_df, validate_df = train_test_split(df, test_size=0.3, random_state=42)
validate_df, test_df = train_test_split(validate_df, test_size=0.33, random_state=42)

In [13]:
import time
start_time = time.time()

In [ ]:
logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

# We use the RoBERTa base pre-trained model.
model = ClassificationModel('bert', 'bert-large-uncased', num_labels=2, args=train_args)

# Train the model, use the validation set as the development set as per the paper.
# When training to 1 epoch this is not that essential, however, if you decide to
# train more and configure early stopping, do check out the simple transformers
# documentation: https://simpletransformers.ai/docs/tips-and-tricks/#using-early-stopping
model.train_model(train_df, eval_df=validate_df)
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
end_time = time.time()
print("Total training time: {}".format(end_time - start_time))

Total training time: 797.3813986778259


In [ ]:
#f1-score
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.f1_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
precision = result["tp"]/(result["tp"]+result["fp"])
print(precision)

0.7995283018867925


In [ ]:
recall = result["tp"]/(result["tp"]+result["fn"])
print(recall)

0.710691823899371


In [ ]:
f1 = 2*((recall*precision)/(recall+precision))

print(f1)

0.7524972253052163


In [ ]:
accuracy = (result["tp"]+result["tn"])/(result["tp"]+result["tn"]+result["fp"]+result["fn"])

print(accuracy)

0.7602150537634409
